# Importing dataset and libaries

In [ ]:
# Import needed libaries
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# read in dataset and remove previouly set multiindex
df = pd.read_pickle("df4_mean_diff_ref_added.pkl")

df

In [ ]:
# Convert dates to datetime and then to mm-yyyy-dd
df['pa_datum'] = pd.to_datetime(df['pa_datum'], format='%Y-%m-%d %H:%M:%S')
df['pa_datum'] = df['pa_datum'].dt.date

# create a 'month_year' column for grouping
df['month_year'] = df['pa_datum'].map(lambda x: x.strftime('%Y-%m'))
df

In [ ]:
df['qc_class'] = np.where(df['qc_salzrckhalt'] > 99.5,'High', 'Low') 

In [ ]:
df['p_product']

In [ ]:
import dash
import plotly
import dash_core_components as dcc
import dash_html_components as html
import dash_auth
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
print(dcc.__version__)
print(dash.__version__)

In [ ]:
# Define a color dictionary
color = {"green": "rgb(149, 192, 66)", "blue": "rgb(0, 172, 214)", "grey": "rgb(156, 157, 159)"}

# app.py

In [ ]:
# plot three values of different period of time

fig = make_subplots(rows = 3, cols = 1, shared_xaxes= False)

fig.append_trace(go.Scattergl(x=df['month_year'], y=df['pa_chlorkonzentration_mean'],
                    mode='markers',
                    name='markers'),
                    row = 1, col = 1)
fig.append_trace(go.Scattergl(x=df['month_year'], y=df['pa_cm-pda_mean'],
                    mode='markers',
                    name='markers'),
                    row = 2, col = 1)
fig.append_trace(go.Scattergl(x=df['month_year'], y=df['qc_salzrckhalt'],
                    mode='markers',
                    name='markers'),
                    row = 3, col = 1)

fig.update_layout(height=800, width=600, title_text="Stacked subplots")
fig.show()

In [ ]:
# Colorlist for the producttypes, for grouping 
colorlist = [ 'aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure',
                'beige', 'bisque', 'black', 'blanchedalmond', 'blue',
                'blueviolet', 'brown', 'burlywood', 'cadetblue',
                'chartreuse', 'chocolate', 'coral', 'cornflowerblue',
                'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan',
                'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen',
                'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange',
                'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
                'darkslateblue', 'darkslategray', 'darkslategrey',
             'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 'navy',
                'oldlace', 'olive', 'olivedrab', 'orange', 'orangered',
                'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise']#,
                #palevioletred, papayawhip, peachpuff, peru, pink,
                #plum, powderblue, purple, red, rosybrown,
                #royalblue, saddlebrown, salmon, sandybrown,
                #seagreen, seashell, sienna, silver, skyblue,
                #slateblue, slategray, slategrey, snow, springgreen,
                #steelblue, tan, teal, thistle, tomato, turquoise,
                #violet, wheat, white, whitesmoke, yellow,
                #yellowgreen]


# Needed callback functions

In [ ]:
def scatter_plot(df,x_feature, y_feature, clickData = None):
    
    i = 0
    j = 0
    traces = []
    for product, producttype in df.groupby(['p_product']): # same as hue in seaborn
        i = i+1
        
        traces.append(go.Scattergl(y=producttype[y_feature], # Scattergl improves perf
                      x=producttype[x_feature], 
                      mode='markers', 
                      marker=dict(size=10,
                                  symbol = "circle",
                                  color = colorlist[i],
                                  line=dict(width=2,
                                            color="white",
                                           ),
                                 ),
                      text = df["p_product"],
                      hovertemplate= "<b>%{text}</b><br>"+x_feature+": %{x}<br>"+y_feature+": %{y}<br>",
                      name = product,
                      selected = dict(marker = dict(color = color["blue"])),
                      unselected = dict(marker = dict(opacity = 0.5)),
                     ))
  
        
        layout = go.Layout(title = dict(text = "{} and {} of Product Type".format(x_feature, y_feature), 
                                   x = 0.5,
                                  ),
                      xaxis = dict(title = "{}".format(x_feature), 
                                   #range = [df[x_feature].min()*0.8, df[x_feature].max()*1.2],
                                   linecolor = "black", 
                                   linewidth = 1,
                                   ticks = "outside",
                                   tickcolor = "black", 
                                   mirror = True,
                                   showgrid = True,
                                   gridcolor = color["grey"],
                                   gridwidth = 1,
                                   zeroline = True,
                                   zerolinecolor = color["grey"],
                                   zerolinewidth = 1,
                                  ),
                      yaxis = dict(title = "{}".format(y_feature), 
                                   #range = [df[y_feature].min()*0.8, df[y_feature].max()*1.2],
                                   linecolor = "black", 
                                   linewidth = 1,
                                   ticks = "outside",
                                   tickcolor = "black", 
                                   mirror = True,
                                   showgrid = True,
                                   gridcolor = color["grey"],
                                   gridwidth = 1,
                                   zeroline = True,
                                   zerolinecolor = color["grey"],
                                   zerolinewidth = 1,
                                  ),    
                      font=dict(
                          family="Verdana, monospace",        
                          size=18,        
                          color="black"
                      ), 
                      paper_bgcolor = "white",
                      plot_bgcolor = "white",
                      hovermode = "closest",
                     )
        
        
        
        
    fig = go.Figure(data = traces, layout = layout)
    # Updating figure size
    #fig.update_layout(
        #autosize = False,
       # width = 500,
        #height = 500,
    #)
    
    if clickData is not None: 
        fig.update_traces(
            overwrite=True,
            selectedpoints = [clickData["points"][0]["pointIndex"]]
            )
            
    return fig


# hot-reloading by enable debug = True, if code changes site will update

In [ ]:
scatter_plot(df,'pa_chlorkonzentration_mean','pa_cm-pda_mean')

# Dashboarding

In [ ]:
#Set up Dashboard
header = 'Data Project'
#define the border
border = None
#border = "thin grey solid"



app = dash.Dash()

#style definition for each html element
app.layout = html.Div([
    html.Div([
        html.H1(children = header),
        ], style = {"border": border, "text-align": "center", "margin-top": 20, "color": "black"}),
    
            html.Div([
                html.Div([
                        html.H2('Feature Correlation',
                               style = {"text-align": "center", "margin-top": 18}),
                ], style = {"border": border, "margin": "1%",}),
            
                # Dropdown menues for x and y axis in scatterplot
                
                html.Div([
                 html.Div([                    
                     html.Label("Select a feature for the x-axis: "),                    
                     html.Div([                        
                         dcc.Dropdown(
                             id = "x_axis",
                             options  = [{"label": i, "value":i} for i in df.columns],
                             value = "pa_chlorkonzentration_mean",),
                     ], style={"border": border, "margin-top": 10, "margin-bottom": 10}),
                 ], style = dict(border = border, width = "47%", display = "inline-block", margin = "1%")),
                 html.Div([ 
                     html.Label("Select a feature for the y-axis:"),
                     html.Div([                        
                         dcc.Dropdown(
                             id="y_axis",
                             options=[{"label": i, "value": i} for i in df.columns],
                             value="pa_cm-pda_mean",),
                     ], style={"margin-top": 10, "margin-bottom": 10}),
                 ], style = dict(border = border, width = "47%", float = "right", display = "inline-block", margin = "1%")),
             ], style = {"border": border, "margin": "1%"},
            ),
         dcc.Graph(id = "solvent_scatter", style = dict(border = border, margin = "1%", height='80vh')),],),
])

# 80vh adjusting to whole window
#The callback that updates the feature plot, when the dropdown is chosen or when a point is selected
@app.callback(
    dash.dependencies.Output("solvent_scatter", "figure"),
    [dash.dependencies.Input("x_axis", "value"),
     dash.dependencies.Input("y_axis", "value"),
     dash.dependencies.Input("solvent_scatter","clickData")]
    )
                
def update_scatter(x_axis,y_axis,clickData):
    plot = scatter_plot(df,x_axis, y_axis, clickData)
    return plot    
   

    
app.run_server(port= 5001)